<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
以下代码为 <a href="http://mng.bz/orYv">《从零开始构建大型语言模型》</a> 一书的补充代码，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>中文翻译和代码详细注释由Lux整理，Github下载地址：<a href="https://github.com/luxianyu">https://github.com/luxianyu</a>
    
<br>Lux的Github上还有吴恩达深度学习Pytorch版学习笔记及中文详细注释的代码下载
    
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 使用 GPT-4 通过反思调优改进指令数据


- 本笔记本使用 OpenAI 的 GPT-4 API 来实现论文 [Reflection-Tuning: Data Recycling Improves LLM Instruction-Tuning](https://arxiv.org/abs/2310.11716) 中的数据集优化过程  

![](https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/reflection-tuning/reflection-tuning.webp)

- 在原论文中，研究人员对 [Alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca) 和 [WizardLM](https://huggingface.co/datasets/WizardLMTeam/WizardLM_evol_instruct_70k) 指令微调数据集进行了优化；在本笔记本中，我们优化了 [第 7 章使用的指令数据集](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch07/01_main-chapter-code/instruction-data.json)（由于其格式与 Alpaca 相同，因此相同的代码也适用于 Alpaca 数据集）

- 期望的数据集格式如下：


```python
    {
        "instruction": "Edit the following sentence for grammar.",
        "input": "He go to the park every day.",
        "output": "He goes to the park every day."
    },
    {
        "instruction": "Convert 45 kilometers to meters.",
        "input": "",
        "output": "45 kilometers is 45000 meters."
    },
```

> 请注意，本笔记本复现了论文中的方法，作者使用 GPT API 来增强现有数据集。但需要注意的是，根据 [OpenAI 使用条款](https://openai.com/policies/row-terms-of-use/) 的规定，GPT API 生成的数据不得用于开发与 OpenAI 竞争的模型：“你不能做的事……使用输出开发与 OpenAI 竞争的模型。”  
你可以在 [此处](https://www.reddit.com/r/LocalLLaMA/comments/17vbg1f/does_openai_tos_prohibit_generating_datasets_for/) 查阅相关讨论。


In [1]:
# pip install -r requirements-extra.txt

In [2]:
from importlib.metadata import version

pkgs = [
    "openai",  # OpenAI API
    "tqdm",    # Progress bar
]

for p in pkgs:
    print(f"{p} version: {version(p)}")

openai version: 1.30.3
tqdm version: 4.66.4


## 测试 OpenAI API


- 首先，让我们测试 OpenAI API 是否正确设置  
- 如果你还没有账户，需要在 https://platform.openai.com/ 创建一个  
- 请注意，由于 GPT-4 API 并非免费，你还需要向账户中充值（详见 https://platform.openai.com/settings/organization/billing/overview）  
- 按照本笔记本中的代码运行，使用 GPT-4o-mini 大约花费 \$0.03（3 美分，截至本文撰写时）  
- 将上述两种方法应用于第 7 章指令数据集中的全部 1100 条条目，大约花费 \$0.60（60 美分）


- 首先，我们需要提供 OpenAI API 的密钥，可在 https://platform.openai.com/api-keys 找到  
- 请确保不要将此密钥分享给任何人  
- 将此密钥（`"sk-..."`）添加到本文件夹下的 `config.json` 文件中


In [3]:
import json
from openai import OpenAI

# Load API key from a JSON file.
# Make sure to replace "sk-..." with your actual API key from https://platform.openai.com/api-keys
with open("config.json", "r") as config_file:
    config = json.load(config_file)
    api_key = config["OPENAI_API_KEY"]

client = OpenAI(api_key=api_key)

- 首先，让我们通过一个简单示例测试 API，确保其按预期工作：


In [4]:
def run_chatgpt(prompt, client, model="gpt-4o-mini", system_prompt=None):
    # Define the system message if a system_prompt is provided
    messages = []
    
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    # Add the user prompt to the messages
    messages.append({"role": "user", "content": prompt})

    # Call the API
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
        seed=123,
    )
    
    # Return the model's response
    return response.choices[0].message.content


prompt = "Respond with 'hello world' if you got this message."
run_chatgpt(prompt, client)

'hello world'

## 加载 JSON 条目


- 接下来，让我们加载并处理指令数据集  
- 在这里，我们假设已将测试数据集和模型响应保存为 JSON 文件，可以按如下方式加载：


In [5]:
from pathlib import Path


json_file = Path("..") / "01_main-chapter-code" / "instruction-data.json"

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 1100


- 我们打印数据集中的一条条目以查看其结构：


In [6]:
from pprint import pp as pprint

pprint(json_data[0])

{'instruction': 'Evaluate the following phrase by transforming it into the '
                'spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the '
           'correct spelling is "friend".'}


## 改进指令


- Reflection-Tuning 的作者提出了两种方法：(1) 改进指令，(2) 改进响应  
- 我们先从改进给定数据集中的指令开始  
- 下面是来自 [Reflection-Tuning 仓库](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py) 的一个小工具函数，用于将输入格式化，以便 GPT-4 模型进行数据集优化


In [7]:
def instr_prompt_no_input(ins, outp):

    sys_prompt = "You are a helpful, precise but picky assistant for checking the quality of a given instruction."
    prompt_template = "[Instruction]\n{ins}\n\n[The Start of Answer]\n{outp}\n\n[The End of Answer]\n\n[System]\n{criteria}\n\n"
    criteria = "We would like you to answer several questions related to the quality of a given instruction. \n" + \
                "1. Why this instruction is not good? First analyse the instruction based on Complexity of the Topic, Level of Detail Required, Knowledge Required, Ambiguity of the Instruction and Logical Reasoning or Problem-Solving Involved. \n" + \
                "Then analyse why this answer is not good for the given instruction? Analyse based on the Helpfulness, Relevance, Accuracy and Level of Details. \n" + \
                "Finally analyse why this bad instruction lead to a bad answer. " +\
                "2. Based on the reason you provided, generate a new and complete instruction which is complex and difficult to answer directly. " + \
                "Make sure the new instruction is relevent but independent to the original instruction, which can be answered without knowing the original instruction, put the new instruction in the format of [New Instruction] your instruction [End]" +\
                "3. Answer the newly generated instruction as detailed as possible, in the format of [New Answer] your answer [End] \n"
    prompt = prompt_template.format(
        ins=ins, outp=outp, criteria=criteria
    )
    return sys_prompt, prompt

- 为了了解其工作原理，请参考数据集条目 `json_data[2]`


In [8]:
pprint(json_data[2])

{'instruction': 'Convert 45 kilometers to meters.',
 'input': '',
 'output': '45 kilometers is 45000 meters.'}


- 我们可以使用上面定义的 `instr_prompt_no_input` 函数按如下方式优化指令：


In [ ]:
entry = json_data[2]

system_prompt, prompt = instr_prompt_no_input(ins=entry["instruction"], outp=entry["output"])
output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)

print(output)

1. **Analysis of the Instruction:**

   - **Complexity of the Topic:** The topic of converting kilometers to meters is relatively simple and straightforward, as it involves basic unit conversion.
   - **Level of Detail Required:** The instruction does not require much detail; it simply asks for a conversion without any additional context or explanation.
   - **Knowledge Required:** Basic knowledge of metric units and their conversions is required, which is common knowledge.
   - **Ambiguity of the Instruction:** The instruction is clear and unambiguous; it specifies exactly what needs to be converted.
   - **Logical Reasoning or Problem-Solving Involved:** There is minimal logical reasoning involved, as the conversion factor (1 kilometer = 1000 meters) is a standard fact.

   **Analysis of the Answer:**

   - **Helpfulness:** The answer is helpful in that it provides the correct conversion.
   - **Relevance:** The answer is relevant to the instruction, as it directly addresses the conv

- 响应内容非常详细，这对于分析非常有用；同时，它还帮助 GPT-4 模型通过思维链（chain-of-thought）提示方法进行改进  
- 然而，在构建改进后的数据集时，我们实际上只关心新的指令和输出，而不是分析内容  
- 我们可以使用来自 [Reflection-Tuning 仓库](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py) 的以下工具代码来提取模型改进后的指令和输出


In [ ]:
import re

def extract_ins(text, no_input=True):
    if '[New Instruction]' in text:
        pattern = r'(\[New Instruction\])(.*?)(\[End\]|\[New Answer\]|New Answer:)'
    else:
        pattern = r'(New Instruction:)(.*?)(\[End\]|\[New Answer\]|New Answer:)'
    segments = re.findall(pattern, text, re.DOTALL)
    if len(segments) == 0:
        seg_ins = ''
    else:
        seg_ins = segments[0][1].strip()
    if seg_ins.endswith("\n\n3."):
        seg_ins = seg_ins[:-4]
    return seg_ins


def extract_oup(text, no_input=True):
    if '[New Answer]' in text:
        pattern = r'(\[New Answer\])(.*?)(\[End\]|$)'
    else:
        pattern = r'(New Answer:)(.*?)(\[End\]|$)'
        # pattern = r'(\[New Answer\]|New Answer:)(.*?)(\[End\]|$)'
    segments = re.findall(pattern, text, re.DOTALL)
    if len(segments) == 0:
        seg_oup = ''
    else:
        seg_oup = segments[0][1].strip()
    return seg_oup


def extract_instruction(text):
    if text == '':
        return []
    seg_ins = extract_ins(text, no_input=True)
    seg_oup = extract_oup(text, no_input=True)
    return [seg_ins, seg_oup]

- 让我们使用这些工具函数，从之前 GPT-4 生成的冗长输出中提取改进后的指令和响应：


In [ ]:
new_instr, new_outp = extract_instruction(output)

In [ ]:
print(new_instr)

Explain the significance of the metric system in global trade and provide examples of how unit conversions can impact international business transactions.


In [ ]:
print(new_outp)

The metric system, also known as the International System of Units (SI), is a decimal-based system of measurement that is used globally. Its significance in global trade lies in its standardization, which facilitates international communication and commerce. 

   One of the primary advantages of the metric system is that it is universally recognized, which reduces confusion and errors in measurement. For example, when a company in the United States imports goods from Europe, the specifications for those goods are often provided in metric units. If the U.S. company is accustomed to using imperial units (like inches or pounds), they must convert these measurements to ensure compatibility. 

   Unit conversions can significantly impact international business transactions. For instance, if a manufacturer orders 100 kilograms of a product but mistakenly interprets it as 100 pounds, they will receive a much smaller quantity than intended, leading to production delays and financial losses. 



- 请注意，目前指令优化仅适用于没有 `"input"` 字段的数据集条目


## 改进响应


- 类似地，我们也可以将 Reflection-Tuning 的优化过程专门应用于数据集的响应（即 "output" 字段）  
- 下面是来自 [Reflection-Tuning 仓库](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py) 的两个小工具函数，用于将输入格式化，以便 GPT-4 模型进行数据集优化


In [ ]:
def res_gen_prompt_no_input(ins, outp):

    sys_prompt = "You are a helpful, precise but picky assistant for checking the quality of the answer to a given instruction."
    prompt_template = "[Instruction]\n{ins}\n\n[The Start of Answer]\n{outp}\n\n[The End of Answer]\n\n[System]\n{criteria}\n\n"
    criteria = "We would like you to answer several questions related to the quality of the answer to the given instruction. \n" + \
                "1. Why this answer is not good for the given instruction? Analyse based on the Helpfulness, Relevance, Accuracy and Level of Details. \n" + \
                "2. Based on the reason you provided, generate a better answer, new and complete, as detailed as possible, in the format of [Better Answer] your answer [End] \n" 
    prompt = prompt_template.format(
        ins=ins, outp=outp, criteria=criteria
    )
    return sys_prompt, prompt


def res_gen_prompt_input(ins, inp, outp):

    sys_prompt = "You are a helpful and precise assistant for checking the quality of the answer to a given instruction and its input."
    prompt_template = "[Instruction]\n{ins}\n\n[The Start of Input]\n{inp}\n\n[The End of Input]\n\n[The Start of Answer]\n{outp}\n\n[The End of Answer]\n\n[System]\n{criteria}\n\n"
    criteria = "We would like you to answer several questions related to the quality of the answer to the given instruction and corresponding input. \n" + \
                "1. Why this answer is not good for the given instruction and corresponding input? Analyse based on the Helpfulness, Relevance, Accuracy and Level of Details. \n" + \
                "2. Based on the reason you provided, generate a better answer, new and complete, as detailed as possible, in the format of [Better Answer] your answer [End] \n" 
    prompt = prompt_template.format(
        ins=ins, inp=inp, outp=outp, criteria=criteria
    )
    return sys_prompt, prompt

- 同样，让我们将其应用于数据集中的一条条目，以查看其工作原理并生成改进后的响应：


In [15]:
entry = json_data[2]

system_prompt, prompt = res_gen_prompt_no_input(ins=entry["instruction"], outp=entry["output"])
output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)

print(output)

1. The answer provided is not good for the given instruction for several reasons:

- **Helpfulness**: While the answer does provide the correct conversion, it lacks any explanation or context. A more helpful answer would include a brief explanation of the conversion process, which would aid understanding.

- **Relevance**: The answer is relevant in that it addresses the instruction to convert kilometers to meters, but it could be more relevant by including the conversion factor used (1 kilometer = 1000 meters).

- **Accuracy**: The answer is accurate in terms of the numerical conversion (45 kilometers = 45000 meters). However, it could be misleading if the reader does not understand how the conversion was derived.

- **Level of Details**: The answer is very brief and lacks detail. A more detailed response would include the conversion factor and a step-by-step explanation of how the conversion is performed.

2. [Better Answer] To convert kilometers to meters, you can use the conversion 

- 如上所示，响应中包含了对原始响应的分析；我们可以使用来自 [Reflection-Tuning 仓库](https://github.com/tianyi-lab/Reflection_Tuning/blob/main/reflection_code/reflect_response.py) 的以下工具函数来提取新的响应


In [16]:
def extract_response(text):
    if text.count('[Better Answer]') >= 2:
        pattern = r'\[(Better Answer)\](.*?)(\[End\]|\[Better Answer\]|$)'
        segments = re.findall(pattern, text, re.DOTALL)
    else:
        # pattern = r'\[(Better Answer)\](.*?)\[End\]'
        pattern = r'\[(Better Answer)\](.*?)(\[End\]|End|$)'
        segments = re.findall(pattern, text, re.DOTALL)
    return [segment[1].strip() for segment in segments]

In [17]:
response = extract_response(output)[0]
print(response)

To convert kilometers to meters, you can use the conversion factor that 1 kilometer is equal to 1000 meters. Therefore, to convert 45 kilometers to meters, you multiply 45 by 1000. 

So, 45 kilometers × 1000 meters/kilometer = 45000 meters. 

Thus, 45 kilometers is equal to 45000 meters.


## 改进数据集


- 现在，让我们将指令反思和响应反思技术应用于实际数据集  
- 注意：这里仅对一个小数据子集进行演示；若要应用于整个数据集，请更改


```python
data_to_process = json_data[:3]
```

to

```python
data_to_process = json_data
```

### 指令反思


- 以下代码将 Reflection-Tuning 方法应用于原始数据集中的指令，以进行数据集优化


In [18]:
data_to_process = json_data[:3]

In [19]:
from tqdm import tqdm


def reflect_instructions(json_data, client):
    new_json_data = [] 
    
    for entry in tqdm(json_data):
        
        if not entry["input"]:
            system_prompt, prompt = instr_prompt_no_input(ins=entry["instruction"], outp=entry["output"])
            output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)
            new_instr, new_outp = extract_instruction(output)
            new_entry = {"instruction": new_instr, "input": "", "output": new_outp}
            new_json_data.append(new_entry)
        else:
            new_json_data.append(entry)

    return new_json_data

In [20]:
data_to_process = json_data[:3]

new_json_data = reflect_instructions(data_to_process, client)

100%|█████████████████████████████████████████████| 3/3 [00:06<00:00,  2.17s/it]


In [21]:
for i in new_json_data[:3]:
    pprint(i)
    print("\n\n")

{'instruction': 'Evaluate the following phrase by transforming it into the '
                'spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the '
           'correct spelling is "friend".'}



{'instruction': 'Edit the following sentence for grammar.',
 'input': 'He go to the park every day.',
 'output': 'He goes to the park every day.'}



{'instruction': 'Explain the significance of understanding metric conversions '
                'in scientific research, and provide an example of how a '
                'miscalculation in unit conversion could impact experimental '
                'results.',
 'input': '',
 'output': 'Understanding metric conversions is crucial in scientific research '
           'because accurate measurements are fundamental to the validity of '
           'experimental results. The metric system is widely used in '
           'scientific disciplines due to its ease of use and universal '
    

- 让我们保存新的数据集：


In [22]:
with open("instruction-reflected.json", "w") as file:
    json.dump(new_json_data, file, indent=4)

### 响应反思


- 现在，让我们对响应反思执行相同操作：


In [23]:
data_to_process = json_data[:3]

In [24]:
def reflect_responses(json_data, client):
    new_json_data = [] 
    
    for entry in tqdm(json_data):
        
        if not entry["input"]:
            system_prompt, prompt = res_gen_prompt_no_input(ins=entry["instruction"], outp=entry["output"])
            output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)
            new_response = extract_response(output)

            if not len(new_response):
                new_response = entry["output"]
                      
            new_entry = {"instruction": entry["instruction"], "input": "", "output": new_response[0]}
            new_json_data.append(new_entry)

        else:
            system_prompt, prompt = res_gen_prompt_input(ins=entry["instruction"], inp=entry["input"], outp=entry["output"])
            output = run_chatgpt(prompt=prompt, client=client, system_prompt=system_prompt)
            new_response = extract_response(output)

            if not len(new_response):
                new_response = entry["output"]

            new_entry = {"instruction": entry["instruction"], "input": entry["input"], "output": new_response[0]}
            new_json_data.append(new_entry)

    return new_json_data

In [25]:
new_json_data = reflect_responses(data_to_process, client)

100%|█████████████████████████████████████████████| 3/3 [00:07<00:00,  2.40s/it]


In [26]:
for i in new_json_data[:3]:
    pprint(i)
    print("\n\n")

{'instruction': 'Evaluate the following phrase by transforming it into the '
                'spelling given.',
 'input': 'freind --> friend',
 'output': 'The input phrase "freind" contains a spelling error. The correct '
           'transformation of the word is as follows: "freind" should be '
           'corrected to "friend." Therefore, the correct spelling is '
           '"friend."'}



{'instruction': 'Edit the following sentence for grammar.',
 'input': 'He go to the park every day.',
 'output': 'The original sentence "He go to the park every day" contains a '
           'grammatical error in the verb form. The correct form should be "He '
           'goes to the park every day." This is because the subject "He" is '
           'third person singular, and in English, the verb "to go" changes to '
           '"goes" when used with third person singular subjects. Therefore, '
           'the corrected sentence is grammatically accurate and maintains the '
           'original mea

- 让我们保存新的数据集：


In [27]:
with open("response-reflected.json", "w") as file:
    json.dump(new_json_data, file, indent=4)

## 创建改进后的指令数据


- 将上述两种方法应用于第 7 章指令数据集中的全部 1100 条条目，大约花费 \$0.60（60 美分）  
- 为了避免在 GitHub 仓库中增加数据集文件，生成的数据集文件可通过 Google Drive 获取：  
  - [instruction-reflected.json](https://drive.google.com/file/d/1c1QnuTdt9nP1u51vBn4_b05mWR_ZNGBv/view?usp=sharing)  
  - [response-reflected.json](https://drive.google.com/file/d/1RNckTZ2ELcdUoJtaylao6NvyZPMtNv1v/view?usp=sharing)
